In [1]:
import os
import datetime
import timedelta
import pandas as pd
import urllib.request, json
from UpdateDerivedValues import updateDerivedValues
# updateDerivedValues


date = (datetime.datetime.now() - timedelta.Timedelta(days=0)).date()
# present = datetime.datetime.now().date()

folders = ["INPUT","RAWCSV","LOG"]
for folder in folders:
    if not os.path.isdir(os.path.join("..","..",folder,str(date))):
        os.mkdir(os.path.join("..","..",folder,str(date)))
        print("Created")

In [2]:
source = pd.read_csv("../../sources.csv")

In [3]:
def getPreviousRawFile(date,state):
    prevDate = date - timedelta.Timedelta(days=1)
    try:
        state_df = pd.read_csv("../../RAWCSV/"+str(prevDate)+"/"+state+"_final.csv")
    except:
        state_df = getPreviousRawFile(prevDate,state)
    return state_df

In [5]:
path = "../../RAWCSV/"+str(date)+"/"
dir_list = os.listdir(path)
for state in source["StateCode"]:
    if state != "TT":
        if state+"_raw.csv" not in dir_list:
            df = getPreviousRawFile(date,state)
            df["Date"] = str(date)
            df.to_csv(path+state+"_final.csv",index=False)
        else:
            updateDerivedValues(state,str(date))

Other States population numbers not found
Other Countries population numbers not found


/home/swiadmin/Incovid19/incovid19/src/Interim_update/UpdateDerivedValues.py:379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_raw_csv["delta{}ForDistrict".format(val)][idx] = value
/home/swiadmin/Incovid19/incovid19/src/Interim_update/UpdateDerivedValues.py:381: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_raw_csv["delta{}ForDistrict".format(val)][idx] = None


Running 7DMA for AP
Running 7DMA for BR
14  Mahindergarh population numbers not found
Running 7DMA for HR
Running 7DMA for HP


/home/swiadmin/Incovid19/incovid19/src/Interim_update/get7DMA.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['7DmaTestedForDistrict'][df['District'] != 'Unknown'] = test_df['7DmaTestedForDistrict'][test_df['District'] != 'Unknown']


Other State population numbers not found
Running 7DMA for KA
Running 7DMA for KL
Running 7DMA for OR
Running 7DMA for TN
Running 7DMA for JK
Running 7DMA for PY


In [26]:
df_intTT = pd.read_csv(path+"TT_raw.csv")

In [27]:
df_intTT["Date"] = str(date)
df_intTT["tested_source_state"] = "https://www.icmr.gov.in/"

In [28]:
value = ["Confirmed","Deceased","Recovered"]
for idx in df_intTT.index:
    stateCode = source[source['StateName'] == df_intTT["District"][idx]]["StateCode"].item()
    df_state = pd.read_csv(path + stateCode + "_final.csv")
    for val in value:
        df_intTT["cumulative{}NumberForDistrict".format(val)][idx] = df_state["cumulative{}NumberForState".format(val)][0]
        
for val in value:
    df_intTT["cumulative{}NumberForState".format(val)] = df_intTT["cumulative{}NumberForDistrict".format(val)].sum()

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
def addVaccination(df,date):
    # df = df.drop(['cumulativeVaccinatedNumberForDistrict', 'cumulativeVaccinated1NumberForDistrict', 'cumulativeVaccinated2NumberForDistrict'], axis=1)
    with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+str(date)) as url:
        data = json.loads(url.read().decode())
    df["cumulativeVaccinated1NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_1"]
    df["cumulativeVaccinated2NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_2"]
    df["cumulativeVaccinatedNumberForState"] = data["topBlock"]["vaccination"]["total_doses"]
    states = []
    vaccinated = []
    vaccinated1 = []
    vaccinated2 = []
    for sta in data['getBeneficiariesGroupBy']:
        if (sta['title'].lower() == "dadra and nagar haveli") or (sta['title'].lower() == "daman and diu"):
            states.append('Dadra and Nagar Haveli and Daman and Diu')
        else:
            states.append(sta['title'])
        vaccinated.append(sta['total'])
        vaccinated1.append(sta['partial_vaccinated'])
        vaccinated2.append(sta['totally_vaccinated'])
    VAC_df = pd.DataFrame(list(zip(states,vaccinated,vaccinated1,vaccinated2))).groupby(0, as_index=False).sum()
    VAC_df = VAC_df.rename(columns={0:"District",1:"cumulativeVaccinatedNumberForDistrict",2:"cumulativeVaccinated1NumberForDistrict",3:"cumulativeVaccinated2NumberForDistrict",})
    df = df.merge(VAC_df,how="left",on="District")
    return df

In [30]:
df_intTT = addVaccination(df_intTT,date)

In [31]:
def getDeltaValues(df,date):
    prevDate = date - timedelta.Timedelta(days=1)
    cs = pd.read_csv("../../RAWCSV/"+str(date)+"/TT_final.csv")
    df["deltaConfirmedForState"] = df["cumulativeConfirmedNumberForState"] - cs["cumulativeConfirmedNumberForState"][0]
    df["deltaDeceasedForState"] = df["cumulativeDeceasedNumberForState"] - cs["cumulativeDeceasedNumberForState"][0]
    df["deltaRecoveredForState"] = df["cumulativeRecoveredNumberForState"] - cs["cumulativeRecoveredNumberForState"][0]
    df["deltaTestedForState"] = df["cumulativeTestedNumberForState"] - cs["cumulativeTestedNumberForState"][0]
    return df

In [32]:
df_intTT = getDeltaValues(df_intTT,date)

In [39]:
df_intTT["cumulativeDeceasedNumberForDistrict"].sum()

473620

In [37]:
df_intTT[["District","cumulativeDeceasedNumberForDistrict"]]

,District,cumulativeDeceasedNumberForDistrict
0,Andaman and Nicobar Islands,129
1,Andhra Pradesh,14505
2,Arunachal Pradesh,282
3,Assam,6183
4,Bihar,1211
5,Chandigarh,1081
6,Chhattisgarh,13619
7,Dadra and Nagar Haveli and Daman and Diu,4
8,Delhi,25177
9,Goa,3533


In [ ]:
# AP
case1 : AP.raw is present >> run update derived value >> log.json

case2 : No AP.raw >> AP_final(Prev Day) and change date >> Do not run(Updatederived Value)

#TT_int
for all state files:
    for a state > Confirmed >> TT_raw_int(District Level)[Recoved,D,T,v1,v2] 
    
StateTotal == sum(DistrictCount)[C,R,D,T,V1,V2]

(TT_final_int)Delta = TT_raw_int - TT_final(day-1)

7dma(TT_final_int,interim = True)

JSON Creations

Other State population numbers not found


/home/swiadmin/Incovid19/incovid19/src/Interim_update/UpdateDerivedValues.py:379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_raw_csv["delta{}ForDistrict".format(val)][idx] = value
/home/swiadmin/Incovid19/incovid19/src/Interim_update/UpdateDerivedValues.py:381: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_raw_csv["delta{}ForDistrict".format(val)][idx] = None


Running 7DMA for KA


/home/swiadmin/Incovid19/incovid19/src/Interim_update/get7DMA.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['7DmaTestedForDistrict'][df['District'] != 'Unknown'] = test_df['7DmaTestedForDistrict'][test_df['District'] != 'Unknown']
